In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import PIL
import glob
import rarfile
import numpy as np
import pandas as pd
from collections import namedtuple
from itertools import product

offset = namedtuple('offset', ['width', 'height',
                               'convertMode',
                               'xmin', 'xmax', 'xoffsets',  
                               'ymin', 'ymax', 'yoffsets' ])
offsets = [
    # 1989
    offset(600, 406, 'LA', 1, 125, [20,165, 370, 468], 1, 125, [142,278]), # Sunday 1989
    offset(640, 189, 'LA', 2, 185, [10,230,445], 2, 185, [0]),    # Weekday 1989
    
    offset(640, 186, 'LA', 2, 185, [10,230,445], 2, 185, [0]),    # Weekday 1990
    offset(640, 196, 'LA', 5, 197, [0,218,436], 3, 195, [0]),      # Weekday 1990
    
    offset(640, 199, 'L', 5, 199, [0,218,436], 3, 197, [0]), # Weekday 2012
    offset(640, 288, 'L', 9, 145, [0, 160, 322, 486], 1,137, [0, 149]), # Weekend 2012
    
]

In [ ]:
pattern = '/Users/ajmendez/data/dilbert/raw/*.rar'
sizes = []
for filename in sorted(glob.iglob(pattern)):
    if any([x in filename for x in ['1989']]):
        continue
        
    print(filename)
    with rarfile.RarFile(filename) as rf:
        files = sorted(rf.namelist())
        files = [f for f in files
                 if not (f.replace('.jpg','-colour.jpg') in files)]
        for f in files:
            img = PIL.Image.open(rf.open(f))
            sizes.append([img.width, img.height])
            
            hasoffset = any([(np.abs(img.width-off.width) < 2) &
                             (np.abs(img.height-off.height) < 2) 
                             for off in offsets])
            if hasoffset:
                continue
            break
    print('Done')
    break

print(img.width, img.height)
plt.imshow(img)

In [ ]:
def convert(image, off):
    plt.figure(figsize=(12,12))
    plt.imshow(image.convert('LA'))
    print(off.xmax-off.xmin)
    print(off.ymax-off.ymin)
    for xoffset in off.xoffsets:
        plt.axvline(xoffset+off.xmin, color='r', lw=2)
        plt.axvline(xoffset+off.xmax, color='r', lw=2)
    for yoffset in off.yoffsets:
        plt.axhline(yoffset+off.ymin, color='r', lw=2)
        plt.axhline(yoffset+off.ymax, color='r', lw=2)
#     for j,(yoffset, xoffset) in enumerate(product(offset.yoffsets,offset.xoffsets)):
#         plt.axvline(xoffset, color='r')
#         plt.axhline(yoffset, color='r')
        

off = offset(640, 196, 'LA', 5, 197, [0,218,436], 3, 195, [0])

convert(img, off)

In [ ]:
NX=NY=128
def carveImages(image, off, outpattern=None, convertMode=None):
    if convertMode is None:
        convertMode = off.convertMode
    for k,(yoffset, xoffset) in enumerate(product(off.yoffsets, off.xoffsets)):
        img = image.crop((xoffset + off.xmin, 
                          yoffset + off.ymin, 
                          xoffset + off.xmax - off.xmin, 
                          yoffset + off.ymax - off.ymin))
        img.thumbnail((NX,NY), PIL.Image.ANTIALIAS)
        if outpattern:
            img.convert(convertMode).save(outpattern.format(j=j))
        else:
            yield img.convert(convertMode)

def plotImages(image, off):
    nx,ny = len(off.xoffsets), len(off.yoffsets)
    fig, axes = plt.subplots(ny, nx, figsize=(nx*3, 
                                      ny*3))
    
    for (img,ax) in zip(carveImages(image, off), axes.flatten()):
        plt.sca(ax)
        plt.imshow(img)

plotImages(img, off)
    

In [ ]:
    for xoffset in 
    xmin,xmax = [9,145]
    ymin,ymax = [1,137]
    xoffsets = [0, 160, 322, 486]
    yoffsets = [0, 149]
    # Remember people read left to right -- top to bottom
    for j,(yoffset, xoffset) in enumerate(product(yoffsets, xoffsets)):
        img = image.crop((xoffset + xmin, 
                          yoffset + ymin, 
                          xoffset + xmax - xmin, 
                          yoffset + ymax - ymin))
        img.thumbnail((NX,NY), PIL.Image.ANTIALIAS)
        img.convert('L').save(outpattern.format(j=j))